# Lab Outline

## Online

#### This document
<font color="#f22" >
You can find the most up-to-date version of this document online [here](https://github.com/mjbright/DOCKER_LABS/blob/master/Docker_Build_Lab.md) or as
<br/>
- a [PDF file](https://raw.githubusercontent.com/mjbright/DOCKER_LABS/master/Docker_Build_Lab.pdf) (http://bit.ly/1QF0XaH)  or as
<br/>
- a [Jupyter](http://www.jupyter.org) notebook at [Docker_Build_Lab](https://github.com/mjbright/DOCKER_LABS/blob/master/Docker_Build_Lab/).
</font>

This notebook is runnable in a Jupyter installation with the bash_kernel installed.

Although that is not the subject of this lab, if you want to create your own environment in which to run this lab with Docker components already installed (and even Jupyter/bash_kernel), refer to the README.md [here](https://github.com/mjbright/DOCKER_LABS/blob/master/)

<br/><br/><br/>
<center><font size=-1 color="#77f">
    <i class="fa-linkedin fa-2x fa"> [mjbright](https://www.linkedin.com/in/mjbright)</i>,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
    <i class="fa-github fa-2x fa"> [mjbright](https://github.com/mjbright)</i>,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
    <i class="fa-twitter fa-2x fa">[@mjbright](http://twitter.com/mjbright)</i>,
</font></center>


<a name="TOP"></a>
## Lab-Description
[TOP](#TOP)


We first need to recuperate the source code examples:

<font color="#f22">
### Lab Start
</font>
Start this lab by first performing the below step:

In [ ]:
## Lab Start:

rm -rf ~/src

cd
git clone https://github.com/mjbright/docker-examples src

./src/START_LAB.sh

Then procede with the following sections:

- [1. Introduction](#intro)
- [2. Basic Docker Builds](#builds)
- [3. Creating Small Images](#small-images)
  - [Creating a small binary with C](#small-c)
  - [Creating a small binary with Go](#small-go)
  - [Creating a toolset Docker image containing several executables](#small-push)
  
- [4. Pushing our image to Docker Hub](#small-multi)
  
- [5. Dockerfile best practices](#build-best-practices)

- [6. Using the official 'Language Stack' images](#lang-stacks)
  - [Using a Language Stack (Node.js)](#medium-node)
  - [Using a Language Stack (Python)](#medium-python)
  
- [7. Using Compose](#compose)
  - [Building complex systems with Compose](#Compose)
  - [Rails example with Compose](#Rails-Example-with-Compose)
  
- [8. Building Docker](#building-docker)
  - [Building Docker with Docker](#Building-Docker-with-Docker)

[References](#References)

<hr/>
  
### Overall description of the lab steps

**NOTE: All lab steps can be considered optional, attendees may perform them in order, or jump to the section of interest to them (to get to the more complicated steps)**


<a name="1.intro" />
# Introduction
## A refresh on Docker concepts

You may want to skip this section if you have already run the introductory lab.

Look at what docker version you are running.
Note that the 'docker version' command reports the local client version as well as the server (docker engine) version.

In [2]:
docker version

Client:
 Version:      1.10.0
 API version:  1.22
 Go version:   go1.5.3
 Git commit:   590d5108
 Built:        Thu Feb  4 19:55:25 2016
 OS/Arch:      linux/amd64

Server:
 Version:      1.10.0
 API version:  1.22
 Go version:   go1.5.3
 Git commit:   590d5108
 Built:        Thu Feb  4 19:55:25 2016
 OS/Arch:      linux/amd64


### Images are image layers

Remember that when we talk of a container image it is really a collection of image layers.

The docker info command provides information about the docker engine, see below.

In [ ]:
docker info

But if we look at the number of containers and images, the number of images it is not the same as provided above.
Why do you think that is?

First let's list the number of running and number of stopped containers

**NOTE: the value on your system will be different**

In [ ]:
# Show the running containers:
docker ps

# Count the number of running containers:
echo
echo "Total number of running containers:"
docker ps | tail -n +2 | wc -l

In [ ]:
# Show all the containers (running or stopped):
docker ps -a

# Count all the containers (running or stopped):
echo
echo "Total number of containers (running or stopped):"
docker ps -a | tail -n +2 | wc -l # Number of stopped and running containers ('tail -n +2' excludes the header line)

We can see that the number of containers reported by docker info correctly reports the number of total containers, running or not

But listing images gives a different value from the 'docker info' value

In [ ]:
# Show the images:
docker images

# Count the images:
echo
echo "Total number of images:"
docker images | tail -n +2 | wc -l

That is because there are many intermediate image layers which are not normally listed.
But we can list those layers using the '-a' option and now we see a number close to the value from 'docker info'.

(We will see later how the 'docker history' command allows us to see how the layers were created).

In [ ]:
# Show all the image layers:
docker images -a

# Count all the image layers:
echo
echo "Total number of image layers:"

docker images -a | tail -n +2 | wc -l  # The number of image layers+1 (inc. header line)

Images can include 1 static binary file or more and can even include a whole distribution.
Launching a container launches a single process within that container - which may in turn span other child processes.

Let us look at an extremely small image to have an idea just how small an executable image can be.
Docker provide an official 'hello-world' image which simply echoes some output to the console.

Let's run that image to see and then investigate the image.
First let's search for the image; we see that the first image is 'hello-world' which is an official build

In [ ]:
docker search hello-world

Let's now run that image

In [ ]:
docker run hello-world

# Note how we see the pulling of the image if not already available locally:

If it took a while to run, this was due to the time needed to download the image before running it - see above.

Try the command a second time to see how it runs instantaneously as there is no need to download the image which already exists locally on the 'docker engine'.

In [ ]:
docker run hello-world

# The second time there is no need to repull the image:

Let us inspect the image.  We see that the file is only 960 bytes large, it must be machine code to print out the text.  So we see that an image can be really very small

In [ ]:
docker images hello-world

We can also inspect the image with the history command to see how it was constructed.

Note that history shows the image layers in reverse order, latest first.

From the below command we can see that the image was created from only 2 image layers.

The image was built simply by copying in a binary executable and then specifying the default command to invoke when the image is run.

In [ ]:
docker history hello-world

In [ ]:
echo
echo "Total size (in bytes) of text in 'hello-world' image:"
docker run hello-world | wc -c

So we see that 801 bytes of that executable is the actual text printed !
So the real program size is roughly 160 bytes (of assembler no doubt)

<a name="builds" />
## Basic Docker Builds
[TOP](#TOP)

#### Dockerfile

Images are built from Dockerfiles which contain a series of commands used to build up a docker image.
Note that each command in the Dockerfile results in a new image layer being created, no matter how trivial the command - even ENV "commands" create a new image layer.

In the following lab we will see how images can be built systematically from a Dockerfile using the 'docker build' command.

#### DockerHub

When we pull an image we pull it from a Docker Registry.  The [DockerHub](https://hub.docker.com/) is a free to use Docker registry allowing to store your own image files (which are publicly available unless you pay for your account) and to pull other image files of other users or officially provided images.

You can create images either by
- building them from a Dockerfile (thus in a **repeatable** manner)
- building them manually by modifying a running container and *'commit'*ing it's state

The DockerHub contains images which may be
- **Automated builds** (built from a git repository)
  - Such builds are usually built from an open-source git repo and so are called **Trusted builds** because the source code is available.
    *Note:* The github repo may contain binary files though
- **Official builds** are builds which are builds provided by partners or by Docker themselves

Other images may exist in the hub but their origin is unknown and so represent a security risk.

It is possible to search the DockerHub, or another Docker Registry, using the 'docker search' command with appropriate options.  Other companies offer their own Docker Registry which may be freely accessible e.g. RedHat, internal to a company e.g. HPE IT, or available as part of a paid for service e.g. IBM or Amazon Web Services ECS.


In [ ]:
cd ~/src/basic

In the ~/test folder create a Dockerfile with the contents shown below (the o/p of the cat command).

For this you may use vi if you are familiar, otherwise the 'nano' text editor is recommended.

Use ctrl-W to write out the file and ctrl-X to quit the editor.

In [ ]:
cat Dockerfile

In [ ]:
ls -altr Dockerfile

We can now build a new image using this dockerfile using the below command where
    
- we specify the current directory as the context for the build (any ADD/COPY or Dockerfile files will be sourced from here) with
    the **'.'** option
- we specify the specific tag to use for the generated image as "*lab/basic*" with

    **-t lab/basic**

In [ ]:
docker build -t lab/basic . 

Note that during the build, the RUN commands are actually run.

They are used to build up this new image.

In this case we echo the 'Python' version string during the build process.

You can see the available options to the build command by issuing
 'docker build --help'

In [ ]:
docker build --help

We can see all the images available using the
 'docker images' command
 
but if there are many, how do we see just our newly-created image?

You can see the available options to the images command by issuing
 'docker images --help'

In [ ]:
docker images --help

So you can see your newly built 'lab/basic' with the following command:

In [ ]:
docker images lab/basic

Note that if you rerun the build command, the build should run faster, you will notice how build steps recognize that this step has already been performed and so will use the image layer already available in the local cache.

Now let us see what happens if we modify our Dockerfile, by inserting a line, such as defining an environment variable.

We will use the same Dockerfile, but this time we will insert an "ENV" line


In [ ]:
cd ~/src/basic/

Now edit the Dockerfile to have the contents as shown below (the o/p of the cat command).

In [ ]:
cat Dockerfile

This time when we build the image we will see that the addition of a line between the "RUN" line and the "CMD" line forces rebuild of subsequent image layers.

**We see 'Using cache' for Step 2 and 3 only**

In [ ]:
docker build -t lab/basic . 

Similarly we can force to not use the cache with the --no-cache option.

This could be useful if we suspect the caching is not working properly due to some external change.

In [ ]:
docker build --no-cache -t lab/basic . 

In [ ]:
docker images lab/basic

<a name="small-images" />
## Creating small images
[TOP](#TOP)

<a name="small-c" />
## Creating a small C Docker image
[TOP](#TOP)

In this example we show how we can create a Docker image from a statically-linked binary.

<font size=+1 color="#77f">
<b>The goal of this step is to show that we do not need an Operating System image for a Docker container.</b>
</font>

All we need is a self-contained binary - i.e. statically linked binary.

Of course a dynamically linked binary could also be used, but in this case it's more complicated as you would have to manually add all it's dependent libraries.  Let's let gcc to do that work for us!

This section comprises 2 things
- A Dockerfile to build our image from a static binary
  Note that it starts with "FROM scratch".
  Scratch is a special 'empty' image
- helloFromDocker.c
  

So first let's build our static binary

In [ ]:
cd ~/src/createTinyC/

# For RHEL/Fedora/Centos only:
# First we must install *glibc-static*
#yum install -y glibc-static

gcc -static helloWorld.c -o helloWorld

ls -alh helloWorld

So we see that this created a binary file of approximately 857kby.

Now let's build our Docker image containing this binary.

You will need to recreate the Dockerfile as follows:

In [ ]:
cat Dockerfile

In [ ]:
docker build -t lab/c_prog .

If we now look at the generated Docker image (below) we see an image of about 877kby.

So whilst this is larger than the 1kby hello-world image (no doubt written in assembler) it is still a very small Docker image which is only 20kbytes larger than the original binary file.

In [ ]:
docker images lab/c_prog

And now let's run that image

In [ ]:
docker run lab/c_prog

In [ ]:
docker history lab/c_prog

<a name="small-go" />
## Creating a small Go Docker image
[TOP](#TOP)

That's fine, but isn't Go taking over the world as a systems language?
Docker, Kubernetes, LXD, Rocket, ... many new tools are being written in Go.

Let's see how we can do the same exercise but building a Go statically-linked binary.

<font size=+1 color="#77f">
<b>The goal of this step is as the previous step (building an image from a single statically-linked binary) but using Go, but also to demonstrate how we can use a Docker image containing a Go compiler, rather than explicitly installing a compiler.</b>
</font>

**NOTE:** We will do this **without** 'installing a Go compiler'



  

In [ ]:
cd ~/src/createTinyGo
cat Dockerfile

**NOW we invoke the golang container** to build our go source code.

The following docker run
- mounts the current directory ($PWD) as /go within the container
- launches a container of the **golang** image which contains the go compiler
- invokes the command "go build -v hello" on the container to build the sources for the "hello.go" code.

The hello.go code is located under src/hello/hello.go.

This is a Go convention.

**NOTE:** The important thing to note here is that the compiler is within the image.  We did not need to install a native Go compiler, we used an image which contains the compiler and by mounting the current directory the container can read the source code and write the executable outside the container.  This is a nice pattern of providing a tool within a container.


In [ ]:
# Compile our Go-code using the Go compiler provided by the 'golang' container:
docker run -it -v $PWD:/go golang go build hello

# Now we have our small static binary compiled
ls -l hello

Now we can build our image including this static binary.

In [ ]:
docker build -t lab/go-hello .

In [ ]:
docker images lab/*

<a name="small-multi"/>
## Creating a toolset Docker image containing several executables
[TOP](#TOP)

Now let's see how we can combine these static binaries into one image.

Let's build a new image derived from the Docker provided 'hello-world' image

<font size=+1 color="#77f">
<b>The goal of this step is to show how we can combine several executables in an image, opening up the possibility of creating a container of tools.</b>
</font>

We will do this without directly 'installing a Go compiler' but by using the official *'golang'* image which includes the Go compiler.

In [ ]:
cd ~/src/toolset

cp ../createTinyC/helloWorld   helloWorld
cp ../createTinyGo/hello       helloWorldGo

ls -altr

Create the Dockerfile with the following contents

In [ ]:
cat Dockerfile

Now build the toolset with these executables included:

In [ ]:
docker build -t lab/toolset ./

If we look at the history of this image we can see the different executables and CMDs which have been added including the original hello-world image.

In [ ]:
docker history lab/toolset

Now we are free to specify which command is to be run.

If we don't specify the command, the last (first in the above history list) will be run (so /helloWorldGo in this case)

In [ ]:
docker run lab/toolset

Or we can explicitly choose the executable to be run, such as the /hello executable of the original "hello-world" image

In [ ]:
docker run lab/toolset /hello

In [ ]:
docker run lab/toolset /helloWorld

In [ ]:
docker run lab/toolset /helloWorldGo

We have seen how we can combine several executables in an image, and we can imagine creating a toolset container in this way (with some more useful executable tools!)

<a name="small-push"/>
## Pushing our image to Docker Hub
[TOP](#TOP)

**Note:** If you have your own account on Docker Hub you may wish to use that for this exercise.

**Otherwise** we will all be using the same account **'dockerlabs'** so you will need to specify a tag which
distinguishes your images from your neighbours.


<font size=+1 color="#77f">
<b>The goal of this step is to demonstrate how we may push an image which we have built to the Docker Hub.</b>
</font>

First we will retag our local image to be unique.
If you are on <font size=+1 color="#d88">podN</font>, then tag with <font size=+1 color="#d88">userN</font>,

e.g. if you are <font size=+1 color="#d88">pod3</font>,

<font size=+1 color="#d88"><br/>&nbsp;&nbsp;&nbsp;&nbsp;
docker tag lab/toolset dockerlabs/toolset:user3
</font>

Notice that we then have 2 toolset images with different tags.

They are otherwise identical (but they could be different) and have the same "IMAGE ID".

In [ ]:
docker tag lab/toolset:latest dockerlabs/toolset:userN
docker images */toolset

First we must login to the Docker Hub.

Ask you instructor for the password to the dockerlabs account.

In [ ]:
docker login -u dockerlabs -p $PASSWORD -e dockerlabs@mjbright.net

Now we may push our image to the public Docker Hub

In [ ]:
docker push dockerlabs/toolset:userN

**NOTE:** The docker search command is not very useful.

and the below command doesn't show us the tags ... and so we don't know if the below image is tagged user1, user2, ...

In [ ]:
docker search dockerlabs/

### Logging on to DockerHub to see your tagged image there

So for this step, log onto DockerHub
    https://hub.docker.com/

In [ ]:
# Ignore this line: it is just to display the image below

curl -s 'http://image.slidesharecdn.com/dockerdemystifiedforsbjug-150918181554-lva1-app6892/95/docker-demystified-for-sb-jug-34-638.jpg' | display

As dockerlabs (dockerlabs AT mjbright.net) with the appropriate password (ask your instructor)

Once logged you should see the dockerlabs/toolset listed, otherwise you can search for it.

Click on the [dockerlabs/toolset](https://hub.docker.com/r/dockerlabs/toolset/) link, then on the [Tags](https://hub.docker.com/r/dockerlabs/toolset/tags/) link and you should now see your tagged image there.

#### Remove any running *'dockerlabs/toolset'* containers on your system

We do this step to make sure we can easily delete your local dockerlabs/toolset:userN image.

These steps could be done by hand through use of 'docker ps' and 'docker ps -a' and picking containers ids corresponding to 'dockerlabs/toolset' containers to use with 'docker stop' and 'docker rm' commands.

The below expressions do this automatically for us.

In [ ]:
IMAGE_NAME=dockerlabs/toolset

echo; echo "Currently running or stopped '$IMAGE_NAME' containers"
docker ps -a --filter=ancestor=$IMAGE_NAME

echo; echo "Stopping any running '$IMAGE_NAME' containers (so we can remove dockerlabs/ image)"
docker stop $(docker ps --filter=ancestor=$IMAGE_NAME) 2>/dev/null

echo; echo "Removing any stopped '$IMAGE_NAME' containers (so we can remove dockerlabs/ image)"
docker rm $(docker ps -a --filter=ancestor=$IMAGE_NAME) 2>/dev/null

echo; echo "There should be no more '$IMAGE_NAME' containers present:"
docker ps -a --filter=ancestor=$IMAGE_NAME

In [ ]:
docker images dockerlabs/*

Note that the following rmi command 'Untags' the image.

This is because it is the same - has the same image id - as our original 'lab/toolset' image.

Removing the dockerlabs/toolset image does not remove the identical 'lab/toolset' image but removes the 'dockerlabs/toolset' tag.

In [ ]:
docker rmi dockerlabs/toolset:userN

In [ ]:
docker images dockerlabs/*

As we have removed ('untagged') the dockerlabs/toolset image, the following run command will download it from the Docker Hub

In [ ]:
docker run dockerlabs/toolset:userN

In [ ]:
docker images dockerlabs/*

In [ ]:
docker run dockerlabs/toolset:userN /helloWorld

In [ ]:
docker run dockerlabs/toolset:userN /hello

<a name="build-best-practices"/>
## Dockerfile best practices
[TOP](#TOP)

<font size=+1 color="#77f">
<b>The goal of this step is to demonstrate certain Dockerfile optimizations.</b>
</font>

- group related commands together using '&&' to reduce image layers
- if temporary files are to be removed


In [ ]:
cd ~/src/build-best-practices
cat Dockerfile

TO be completed ... !!

<a name="lang-stacks" />
## Using the official 'Language Stack' images
[TOP](#TOP)

<a name="medium-node" />
## Creating a Node.js application from the Node.js 'LanguageStack' Docker image
[TOP](#TOP)

Docker provide a set of *'Language Stacks'* which are medium sized images representing the necessary dependencies for a particular language.

<font size=+1 color="#77f">
<b>The goal of this step is to demonstrate the use of Docker-provided *Language Stacks*.</b>
</font>

On the [Docker Hub](https://hub.docker.com/) we can find language stacks available for a variety of languages/environments, each with different release versions (Python 2.x and Python 3.x for example):
- [Node.js (Javascript)](https://hub.docker.com/_/node/)
- [Python](https://hub.docker.com/_/python/)
- [Ruby](https://hub.docker.com/_/ruby/)

You can browse the complete list of *'Official Images*' on the Docker Hub [here](https://hub.docker.com/explore/)

Now let's look at an example of Node.js.
To run a Node.js application this time we will need



In [ ]:
docker pull node

In [ ]:
docker images node

In [ ]:
docker history node

In [ ]:
cd ~/src/nodeJS/
ls -altr

Once again edit the Dockerfile to have the contents shown below:

In [ ]:
cat Dockerfile

Now let's build  the image

In [ ]:
docker build -t node-hello .

and run the image in the background, exposing port 80

In [ ]:
docker run -p 80:80 --name web -d node-hello

Now let's use curl to access this container (default port for curl is 80)

In [ ]:
curl http://localhost

<a name="medium-python"/>
## Creating a Python application from the Python 'LanguageStack' Docker image
[TOP](#TOP)

<font size=+1 color="#77f">
<b>The goal of this step is to demonstrate the use of the Python *Language Stack*.</b>
</font>

Now let's look at a Python example.
To run a Node.js application this time we will need


Let's pull and examine the official 'Docker Language Stack' image of Python

Note how the earliest image layers (at the bottom of the list) have the same image ids
as the earliest image layers of the Node;js image.

So we can see that they were both created from the same base.

In [ ]:
docker pull python

In [ ]:
docker images python

In [ ]:
docker history python

The following command will run the 'python' docker image, invoking the command 'python --version' to show us which default python version is available.

In [ ]:
docker run python python --version

Now let's look at extending this to a small 2-tier application with Flask web server as front-end and Redis as a backend database (counter) to count access to a web page.

To run this application create a Dockerfile with the following contents in the 'python_flask' directory

In [ ]:
cd ~/src/python_flask
cat Dockerfile

In [ ]:
docker build -t lab/python_flask .


In [ ]:
docker images lab/*

Now let's run this container in the background

First we must launch our redis DB as a background container, it will be used to count the number of page accesses.

For this we launch a container instance of the 'redis' image, as a background process (-d) and we also give the instance the name 'redis' also. This will allow our 'flask' container to link to this image

In [ ]:
docker run -d --name redis redis

Now we launch our 'flask' image which will be our web server.

We also launch that container in the background.  Note that we link to the 'redis' container.  This allows the 'flask' container to connect to the redis database to get and increment the counter value

In [ ]:
docker run -d --link redis:db -p 5000:5000 lab/python_flask

Now as we make successive curl (or web browser) requests to our 'flask' web server we see that page access counter gets incremented

In [ ]:
curl http://localhost:5000

In [ ]:
curl http://localhost:5000

In [ ]:
curl http://localhost:5000

In [ ]:
curl http://localhost:5000

  <a name="Compose" />
#   Using Compose
<a href="#TOP">TOP</a>
  
#   Building complex systems with Compose
<a href="#TOP">TOP</a>

In [ ]:
cd ~/src/compose

Create a docker-compose.yml specification file with the following contents.

This is a 2-tier architecture with 

In [ ]:
cat docker-compose.yml

Let's look at the docker-compose options

In [ ]:
docker-compose

Let's first do some cleanup of any existing containers invoked by compose

In [ ]:
docker-compose stop

In [ ]:
docker-compose rm -f

Remove any other containers which might be listening on port 80 - here we stop and remove all containers to be sure

In [ ]:
docker stop $(docker ps -q)
docker rm $(docker ps -aq)

Now we will startup our system.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;docker-compose up

will startup the containers specified in docker-compose.yml (first building any images where 'build:' is specified)

Launching in the foreground will produce the following output:

```
Creating compose_webc_1
Building weba
Step 1 : FROM node
 ---> baa18fdeb577
Step 2 : ADD src/ /src
 ---> Using cache
 ---> 6d6e1fb506d4
Step 3 : WORKDIR /src
 ---> Using cache
 ---> f293e728d28f
Step 4 : RUN npm install
 ---> Using cache
 ---> 16107856389c
Step 5 : EXPOSE 80
 ---> Using cache
 ---> eed510133b6b
Step 6 : CMD node index.js
 ---> Using cache
 ---> 2e6df94ecab1
Successfully built 2e6df94ecab1
Creating compose_weba_1
Creating compose_haproxy_1
Attaching to compose_webb_1, compose_webc_1, compose_weba_1, compose_haproxy_1
webb_1    | Running on http://localhost
webc_1    | Running on http://localhost
weba_1    | Running on http://localhost
haproxy_1 | [WARNING] 038/165945 (1) : Server aj_backends/weba is UP, reason: Layer7 check passed, code: 200, info: "HTTP status check returned code <3C>200<3E>", check duration: 6ms. 3 active and 0 backup servers online. 0 sessions requeued, 0 total in queue.
```

where we see that the necessary images were built and then started.
We also see startup of the haproxy container and it's checks that it can contact the web servers.

Let's start the containers in the background

In [ ]:
docker-compose up -d

Now we can see that appropriate images have been built and we can also use 'docker-compose ps' to see the running elements of our system

In [ ]:
docker images compose*

In [ ]:
docker-compose ps

Now we can use curl connecting to localhost:80 (on which the haproxy container is listening) and we will see how haproxy is doing roundrobin scheduling to each of our web containers

In [ ]:
curl localhost:80

In [ ]:
curl localhost:80

In [ ]:
curl localhost:80

Note how the container id changes each time.

Now we can try scaling one of our services as shown below:

In [ ]:
docker-compose scale weba=5

With 'docker-compose ps' we see that we now have scaled to 5 'weba' servers:

In [ ]:
docker-compose ps

In [ ]:
curl localhost:80
curl localhost:80
curl localhost:80
curl localhost:80
curl localhost:80
curl localhost:80

Now let's stop our current system

In [ ]:
docker-compose stop
docker-compose rm -f
docker-compose ps

<a name="building-docker" />
# Building Docker
<a href="#TOP">TOP</a>

<a name="Building-Docker-with-Docker" />
# Building Docker with Docker
<a href="#TOP">TOP</a>

A major advantage of Docker is to simplify build environments.

Let's look at how we can build the Docker engine client/daemon binary without having to explicitly install a development environment.

<font size=+1 color="#77f">
<b>The goal of this step is simply to show the ease with which we can build Docker, thanks to Docker itself.</b>   
</font>

We do not make particular use of the built image.

The process involves the following steps, several of which have already been performed so as to prevent excessive network utilisation during the lab.  Nevertheless all steps are  described here so that you can see just how easy it is to build Docker from scratch:
- Install make
- Clone the Docker source code
- Checkout the same code revision as our current Docker binary (client and daemon)
- Build the code - which pulls the docker-dev image containing the required version of the Go compiler
- Run the executable to demonstrate it is correct

#### Installing make

In your environment we have already installed the make package, but no compiler using yum:

    yum install make

#### Cloning the Docker source code

We have already downloaded the Docker source code from github as follows:

    mkdir -p /root/src/docker
    cd /root/src/docker
    git clone https://github.com/docker/docker .

To build Docker we simply have to build using the

    make build
    
command.

#### Checkout the source code revision corresponding to our installed Docker Engine

If we build the latest sources this may not be compatible with our installed Docker version.

This is the case.  We have 1.10.0-rc2 installed, which has API version 22, but the current github source is 1.10.0-dev which has changed to API version 23.  So if we build this we find that we cannot use this client to communicate with the installed daemon.

So let's checkout the code for 1.10.0-rc2.

At the time of writing this is the latest release(candidate) of the Docker engine.
We can obtain that version of the source code by referring to the releases page https://github.com/docker/docker/releases
and selecting the SHA1 hash of build 1.10.0-rc2 

    git checkout c1cdc6e



#### Build the code - which pulls the docker-dev image containing the required version of the Go compiler

We can build the code as follows:

    make build
    
We have run 'make build' already, so the docker-dev image has already been downloaded (again to prevent excessive network traffic).  The docker-dev image includes the required go compiler and other build tools.

Run 'make build' again and you will see a standard build process and finally where it places the compiled binary

#### Run the executable to demonstrate it is correct

In preparation for the lab we built from the latest source (not the c1cdc6e version we checked out).

Run this build as follows to see that it is not compatible with the installed binary (/usr/bin/docker).
We see that this binary has version 1.10.0-dev and API version 1.23 but that this cannot communicate with our installed binary which has API version 1.22.

In [ ]:
cd /root/src/docker; ls -altr bundles/1.10.0-dev/binary/docker-1.10.0-dev; ./bundles/1.10.0-dev/binary/docker version

But if we run our new build - as follows - created from revision c1cdc6e of the source code (corresponding to Docker version 1.10.0-rc2) we see that it has the correct version, with the same API version and can interrogate the server.


In [ ]:
cd /root/src/docker; ls -altr bundles/1.10.0-rc2/binary/docker-1.10.0-rc2; ./bundles/1.10.0-rc2/binary/docker version

# References
[TOP](#TOP)

- [Dockerfile Reference](https://docs.docker.com/engine/reference/builder/)

- [Compose file documentation](https://docs.docker.com/compose/compose-file/)

- [Compose file reference](https://github.com/docker/compose/blob/1.6.0-rc1/docs/compose-file.md)

- [Visualizing Docker Containers and Images](http://merrigrove.blogspot.in/2015/10/visualizing-docker-containers-and-images.html)

- [Awesome Docker](https://github.com/veggiemonk/awesome-docker)

- [Docker Cheat Sheet]()

- [Building Good Docker Images](http://jonathan.bergknoff.com/journal/building-good-docker-images)

- [How to scale a Docker Container with Docker Compose](https://www.brianchristner.io/how-to-scale-a-docker-container-with-docker-compose/)

- [Docker Compose Demo](https://github.com/vegasbrianc/docker-compose-demo)
